In [1]:
import warnings
import sys 
if not sys.warnoptions:
    warnings.simplefilter("ignore")
import os 
import glob
from scipy import stats
import time
from copy import deepcopy
import numpy as np
import pandas as pd 
from os.path import join as pjoin
from nilearn import datasets, image
from nilearn import surface
from nilearn import plotting
from nilearn.image import load_img
from brainiak.funcalign.srm import SRM
from nilearn.masking import apply_mask, unmask
from nilearn.input_data import NiftiMasker, NiftiLabelsMasker,NiftiMapsMasker
import nibabel as nib
import pickle
import csv
import time
from scipy import signal
#from brainiak import image, io
from brainiak.isc import isc, isfc, permutation_isc
import matplotlib.pyplot as plt
import seaborn as sns 
from brainiak.fcma.util import compute_correlation
from sklearn.svm import NuSVC
from sklearn.metrics import confusion_matrix
%autosave 5
%matplotlib inline

Autosaving every 5 seconds


In [2]:
"""for all the subjects"""
def grab_subject_ids(ds_dir,testsubs=False):
    """
    Get list of all subject IDs.
    """
    import os
    import glob
    sub_ids = [os.path.basename(subdir) for subdir in glob.glob(ds_dir + '/*')]
    if testsubs:
        sub_ids = sub_ids[:testsubs]
    return sub_ids
def datagrabber(roi_glm_workdir='/Volumes/Avinash PhD/cSRM/sense_data',
                prepped_dsdir='/Volumes/Avinash PhD/cSRM/sense_data',
                testsubs=False,
                sub_ids=True):
    """
    # grab file names for
    # filtered bold data and roi masks from roi_glm output
    """
    sub_ids = grab_subject_ids(ds_dir ='/Volumes/Avinash PhD/somato_align/age/old')
    sub_ids.remove('yhx875')
    run1_data, run2_data = [], []
    #run1_masks, run2_masks= [], []
    run1_masks = list(np.zeros(len(sub_ids)))
    for indx,sub_id in enumerate(sub_ids):
        fname = pjoin(roi_glm_workdir, sub_id, 'coreg1.nii.gz')
        fname2 = pjoin(roi_glm_workdir, sub_id, 'coreg2.nii.gz')
        mask_dir = pjoin(prepped_dsdir,sub_id,'label_vol')
        masks=[]
        for lab in labels:
            mask_fname = pjoin(mask_dir, '%s.nii' %lab)
            masks.append(mask_fname)
            
        run1_data.append(fname)
        run2_data.append(fname2)
        run1_masks[indx]= masks
        
    return run1_data, run2_data, run1_masks, sub_ids

In [3]:
"""importing labels"""


with open('/Volumes/Avinash PhD/cSRM/label_list.txt') as inf:
    reader = csv.reader(inf, delimiter="\t")
    labels = list(zip(*reader))[0]

In [4]:
labels

('lh.BA1.label',
 'lh.BA2.label',
 'lh.BA3a.label',
 'lh.BA3b.label',
 'lh.BA4a.label',
 'lh.BA4p.label',
 'lh.BA6.label',
 'rh.BA1.label',
 'rh.BA2.label',
 'rh.BA3a.label',
 'rh.BA3b.label',
 'rh.BA4a.label',
 'rh.BA4p.label',
 'rh.BA6.label')

In [5]:
#sub_ids = grab_subject_ids(ds_dir ='/media/predator/Avinash PhD1/somato_align/age/young')
run1_data, run2_data, run1_masks , sub_ids=datagrabber()

In [ ]:
np.asarray(run1_masks).shape

In [ ]:
masks = run1_masks[1]

In [ ]:
masks[1]

In [ ]:
sub_ids


In [ ]:
"""only for run1 data: 
load the labels in an array corresponding to lists, and also match the affine"""
start_time = time.time()
all_labels2 = list(np.zeros(len(sub_ids)))
for ids,sub in enumerate(sub_ids):
    brain =nib.load(run1_data[ids])
    masks = run1_masks[ids]
    label_arrs = list(np.zeros(14))
    mas = masks[1]
    atlas_map = nib.load(mas)
    br2 = image.resample_to_img(brain,atlas_map)
    for ind,mask in enumerate(masks):
        #atlas_map = nib.load(mask)
        #br2 = image.resample_to_img(brain,atlas_map)
        label_arrs[ind] = apply_mask(br2, mask_img=mask)
    all_labels2[ids] = label_arrs 
    
seconds = time.time() - start_time
print('Time Taken:', time.strftime("%H:%M:%S",time.gmtime(seconds)))


In [ ]:
mask = run1_masks[1][3]

In [ ]:
brain = run1_data[1]

In [ ]:
mask

In [ ]:
atlas_map = nib.load(mask)
br2 = image.resample_to_img(brain,atlas_map)

In [ ]:
nib.save(br2, 'resampled.nii')

In [ ]:
os.system('spd-say "fuck you"')

In [ ]:
with open('all_sub_labels_run1_coreg_per_old.pkl', 'wb') as f:
    pickle.dump(all_labels2, f)

In [7]:
"""only for run2 data: 
load the labels in an array corresponding to lists, and also match the affine"""
start_time = time.time()
all_labels2 = list(np.zeros(len(sub_ids)))
for ids,sub in enumerate(sub_ids):
    brain =nib.load(run2_data[ids])
    masks = run1_masks[ids]
    label_arrs = list(np.zeros(14))
    mas = masks[1]
    atlas_map = nib.load(mas)
    br2 = image.resample_to_img(brain,atlas_map)
    for ind,mask in enumerate(masks):
        #atlas_map = nib.load(mask)
        #br2 = image.resample_to_img(brain,atlas_map)
        label_arrs[ind] = apply_mask(br2, mask_img=mask)
    all_labels2[ids] = label_arrs
    
seconds = time.time() - start_time
print('Time Taken:', time.strftime("%H:%M:%S",time.gmtime(seconds)))



Time Taken: 01:54:12


In [ ]:
label_arrs[13].shape

In [ ]:
labels

In [ ]:
mask

In [ ]:
unm_try=unmask(X=label_arrs[13], mask_img=mask, order='F')

In [ ]:
unm_try.orthoview()

In [ ]:
os.system('spd-say "fuck you"')

In [8]:
with open('all_sub_labels_run2_coreg_per_old.pkl', 'wb') as f:
    pickle.dump(all_labels2, f)